In [ ]:
import datetime as dt
import functools
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from scipy import stats


In [ ]:
# Configuration

raw_data_dir = Path("../../raw_data")
reports_dir = Path("../../reports")
season_id = 25
week_date = dt.datetime(2019, 10, 31)
season_raw_data_dir = raw_data_dir / f"season{season_id}"
season_reports_dir = reports_dir / f"season{season_id}"
season_box_scores_dir = raw_data_dir / f"season{season_id}" / "box_scores"

pd.options.display.float_format = "{:,.2f}".format

In [ ]:
possessions = pd.read_csv(season_raw_data_dir / "possessions.csv")

In [ ]:
player_logs_raw = pd.read_csv(season_raw_data_dir / "player_logs.csv")
player_logs = player_logs_raw.groupby(["game_id", "team_id", "player_id"])

# player_logs.get_group(("20191017-1845-hotshots-rockets", "hotshots", "hotshots-harrison"))["stl"]

In [ ]:
summary_rows = []
for (game_id, team_id), group in possessions.groupby(["game_id", "team_id"]):
    row = {
        "game_id": game_id,
        "team_id": team_id,
        "possessions": group["game_id"].size,
        "players_involved": group["players_involved"].mean(),
        "points_scored": group["points_scored"].mean(),
        "points_after_turnover": group["points_after_turnover"].mean(),
        "second_chance_points": group["second_chance_points"].mean(),
        "passes": group["passes"].mean(),
        "oreb": group["oreb"].mean(),
        "fta": group["fta"].mean(),
    }
    summary_rows.append(row)

summary = pd.DataFrame(summary_rows)
summary


In [ ]:
# Possession finisher / decision maker stat.

display(Markdown("""

### Possession Resolver / Decision Maker

At first, the idea was to calculate how many points on average are scored in possessions 
that are ended by a particular player. If a player shoots and makes the shot,
or shoots and the defense gets the rebound, or turns the ball over, they have effectively
ended their team's possession. Without any weighting added, this stat is pretty interesting itself.
It penalises turnovers and missed shots that end with defensive rebounds, but does not penalise
missed shots that end with offensive rebounds which is usually deemed a good shooting decision.
You can see this stat in the **points_scored** column.

Then, we thought, well, if a player steals the ball and creates possessions, they should not 
be penalised for all turnovers. For example, if a player has 4 steals and 1 turnover then that
one possession ending with a turnover should not count against the player. If, however, the player
has 4 steals and no turnovers, they don't get any discounts. If the player has 1 steal and 4 turnovers
then we can only discount one of their turnovers. This adjustment boosted stats for those who collect
many steals, but since we were now rewarding steals, why not additionaly penalise those who turn the ball
over based on how many points their turnovers cost the team? So we did.

**points_conceded_after_tov** tells how many points did the opposition score in total across all 
possessions following the player's turnover.

This is still not perfect. We are thinking about weighing the steals and turnovers by the number of possessions.

For example 11 ended possessions out of which 4 were turnovers is probably way worse than 4 turnovers 
with 26 ended possessions.

"""))

filter_no_tovs = possessions["resolution_is_tov"] == False

summary_rows = []
for (game_id, team_id, last_player_id), group in possessions[filter_no_tovs].groupby(["game_id", "team_id", "last_player_id"]):
    player_game_stats = player_logs.get_group((game_id, team_id, last_player_id))

#     # If a player has 6 steals and 2 turnovers then we can discount 2 turnovers.
#     # If a player has 2 steals and 6 turnovers then we can discount 2 turnovers.
#     discounted_tov_possessions = min(player_game_stats["tov"].sum(), player_game_stats["stl"].sum())
    
#     points_conceded_after_tov = group[group["resolution_is_tov"]]["opp_points_scored_next"].sum()
    
    row = {
        "game_id": game_id,
        "team_id": team_id,
        "last_player_id": last_player_id,
        "ended_total": group["game_id"].size,
#         "ended_with_tov": group[tovs].count(),
#         "ended_with_tov": group[tovs]["group_id"].size,
#         "points_scored_weighted": (
#             (group["points_scored"].sum() - points_conceded_after_tov)
#             /
#             max(1.0, group["game_id"].size - discounted_tov_possessions)
#         ),
        "points_scored": group["points_scored"].mean(),
        "stl": player_game_stats["stl"].sum(),
        "tov": player_game_stats["tov"].sum(),
#         "discounted_tov_possessions": discounted_tov_possessions,
        "points_conceded_after_tov": points_conceded_after_tov,
    }
    summary_rows.append(row)

summary = pd.DataFrame(summary_rows)
# summary_sorted = summary.sort_values(["points_scored_weighted"], ascending=[False])
summary_sorted = summary.sort_values(["points_scored"], ascending=[False])
summary_sorted = summary_sorted.reset_index(drop=True)
summary_sorted.index += 1
summary_sorted


In [ ]:
# # TODO: Points scored in the possessions starting with your steals
# # TODO HeldBall not handled properly...
# is_after_steal = possessions["first_event_type"] == "Steal"
# summary_rows = []
# for (game_id, team_id, player_id), group in possessions[is_after_steal].groupby(["game_id", "team_id", "first_on_court_player_id"]):
#     player_game_stats = player_logs.get_group((game_id, team_id, player_id))
#     row = {
#         "player_id": player_id,
#         "team_id": team_id,
#         "stl": player_game_stats["stl"].sum(),
#         "points_scored_total": group["points_scored"].sum(),
#         "points_scored_avg": group["points_scored"].mean(),
#         "tov": player_game_stats["tov"].sum(),
#     }
#     summary_rows.append(row)

# summary = pd.DataFrame(summary_rows)
# min_requirements = summary["stl"] > 0

# summary_sorted = summary[min_requirements].sort_values(["points_scored_avg", "points_scored_total"], ascending=[False, False])
# summary_sorted.reset_index(drop=True, inplace=True)
# summary_sorted.index += 1
# summary_sorted

In [ ]:
# # TODO: Points scored in the possessions starting with your steals
# # TODO HeldBall not handled properly...
# is_after_steal = possessions["first_event_type"] == "Steal"
# summary_rows = []
# for (game_id, team_id, player_id), group in possessions[is_after_steal].groupby(["game_id", "team_id", "first_on_court_player_id"]):
#     player_game_stats = player_logs.get_group((game_id, team_id, player_id))
#     row = {
#         "player_id": player_id,
#         "team_id": team_id,
#         "stl": player_game_stats["stl"].sum(),
#         "points_scored_total": group["points_scored"].sum(),
#         "points_scored_avg": group["points_scored"].mean(),
#         "tov": player_game_stats["tov"].sum(),
#     }
#     summary_rows.append(row)

# summary = pd.DataFrame(summary_rows)
# min_requirements = summary["stl"] > 0

# summary_sorted = summary[min_requirements].sort_values(["points_scored_avg", "points_scored_total"], ascending=[False, False])
# summary_sorted.reset_index(drop=True, inplace=True)
# summary_sorted.index += 1
# summary_sorted